# System of Linear Equations

## Question 1

In the classroom we have seen the gauss elimination algorithm,

Apply the algorithm by hand to find the solution to the problem
$Ax=b$ with:

$$A = \begin{bmatrix} 4 & -2 & 1 \\ -2 & 4 & -2 \\ 1 & -2 & 4 \end{bmatrix}$$

$$b^T = \begin{bmatrix} 11 & -16 & 17 \end{bmatrix}$$


Can you write down the algorithm in python ?
You can assume for now that the matrix is nice:
- all the pivot are not null
- there is no need to perform partial pivoting
- back substitution is smooth (none of the pivot are null)


In [1]:
import numpy as np

"""
This implementation closely follows the algorithm
"""
def gaussEliminEasy(a, b):
    n = len(b)
    # Elimination Phase
    for k in range(0, n): #iterate on rows
        b[k] = b[k] / a[k, k]
        a[k, k:n] = a[k, k:n] / a[k, k]
        for i in range(k+1, n): 
            if a[i,k] != 0.0:
                b[i] = b[i] - a[i, k] * b[k]
                a[i, k:n] = a[i, k:n] - a[i, k] * a[k, k:n]
                
    
    print("After forward substitution: ")
    print("A = ")
    print(a)
    print("b = ")
    print(b)
    print()
        
    # Back substitution
    for k in range(n-1, -1, -1):
        for i in range(k-1, -1, -1):
            b[i] = b[i] - a[i, k] * b[k]
            a[i, k:n] = a[i, k:n] - a[i, k] * a[k, k:n]
        
    print("After backward substitution: ")
    print("A = ")
    print(a)
    print("b = ")
    print(b)
    print()
    return b



"""
More advanced implementation
"""
def gaussElimin(a, b):
    n = len(b)
    # Elimination Phase
    for k in range(0, n-1): #iterate on rows
        for i in range(k+1, n): 
            if a[i,k] != 0.0:
                lam = a[i, k] / a[k, k]
                a[i, k+1:n] = a[i, k+1:n] - lam*a[k, k+1:n]
                b[i] = b[i] - lam*b[k]            
        #a[k+1:,k] = 0 #optional, set element bellow pivot to 0
        
    # Back substitution
    for k in range(n-1, -1, -1):
        b[k] = (b[k] - np.dot(a[k, k+1:n], b[k+1:n])) / a[k, k]
    return b



print("Init:")
A = np.array([[4,-2,1], [-2,4,-2], [1,-2,4]], dtype=np.float32)
print("A = ")
print(A)

b = np.array([11,-16,17], dtype=np.float32)
print("b = ")
print(b)
print()

gaussEliminEasy(A, b)

print("Result 2nd implementation: ")
print(gaussElimin(A, b))

Init:
A = 
[[ 4. -2.  1.]
 [-2.  4. -2.]
 [ 1. -2.  4.]]
b = 
[ 11. -16.  17.]

After forward substitution: 
A = 
[[ 1.   -0.5   0.25]
 [ 0.    1.   -0.5 ]
 [ 0.    0.    1.  ]]
b = 
[ 2.75 -3.5   3.  ]

After backward substitution: 
A = 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
b = 
[ 1. -2.  3.]

Result 2nd implementation: 
[ 1. -2.  3.]


## Question 2

Extend the algorithm to simultaneously solve the system for multiple $b$ vectors.
Hint, you can replace the input vector $b$ (dimension $n \times 1$) by a matrix $B$ (dimension $n \times m$)

Let's use the same matrix $A$:
$$A = \begin{bmatrix} 4 & -2 & 1 \\ -2 & 4 & -2 \\ 1 & -2 & 4 \end{bmatrix}$$

And the following $b$ vectors:
$$b_1^T = \begin{bmatrix} 11 & -16 & 17 \end{bmatrix}$$

$$b_2^T = \begin{bmatrix} 11 & -20 & 17 \end{bmatrix}$$

$$b_3^T = \begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$$

or more efficiently with
$$B = \begin{bmatrix} 11 & 11 & 1 \\ -16 & -20 & 1 \\ 17 & 17 & 1 \end{bmatrix}$$



**Solution**

The former code will also work for this case. Indead, instead of considering each element of the $b$ vector, we now need to consider each line of the $B$ matrix. In numpy, 'b[i]' automatically corresponds to the $i$th line in matrix 'b', so no change is required.

## Question 3

Use this simultaneous solver to compute the inverse of the matrix A

Verify that $A^{-1} \times A = I$

In [52]:
A = np.array([[4,-2,1], [-2,4,-2], [1,-2,4]], dtype=np.float32)
b1 = np.array([1,0,0], dtype=np.float32)
b2 = np.array([0,1,0], dtype=np.float32)
b3 = np.array([0,0,1], dtype=np.float32)

x1 = gaussElimin(A.copy(), b1.copy())
x2 = gaussElimin(A.copy(), b2.copy())
x3 = gaussElimin(A.copy(), b3.copy())

Ai = np.array([x1,x2,x3])

print(Ai)
print()
print(np.matmul(Ai,A))

[[0.33333334 0.16666667 0.        ]
 [0.16666666 0.41666666 0.16666667]
 [0.         0.16666667 0.33333334]]

[[ 1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-1.4901161e-08  9.9999988e-01  5.9604645e-08]
 [ 0.0000000e+00  0.0000000e+00  1.0000000e+00]]


## Question 4

Improve your algorithm by adding partial row pivoting
and use it solve the system defined by:

$$A = \begin{bmatrix} 0 & -2 & 2 \\ -2 & 0 & -20 \\ 10 & -2 & 4 \end{bmatrix}$$

$$b^T = \begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$$


In [74]:
import numpy as np

def gaussEliminPP(a, b):
    n = len(b)
    # Elimination Phase
    for k in range(0, n-1): #iterate on rows
        pivot_idx = k + list(np.abs(a[k:n, k])).index(max(np.abs(a[k:n, k])))
        temp_a = a[k].copy()
        temp_b = b[k].copy()
        a[k] = a[pivot_idx]
        b[k] = b[pivot_idx]
        a[pivot_idx] = temp_a
        b[pivot_idx] = temp_b
        
        for i in range(k+1, n): 
            if a[i,k] != 0.0:
                lam = a[i, k] / a[k, k]
                a[i, k+1:n] = a[i, k+1:n] - lam*a[k, k+1:n]
                b[i] = b[i] - lam*b[k]            
        #a[k+1:,k] = 0 #optional, set element bellow pivot to 0
        
    # Back substitution
    for k in range(n-1, -1, -1):
        b[k] = (b[k] - np.dot(a[k, k+1:n], b[k+1:n])) / a[k, k]
    return b
        

A = np.array([[0,-2,2], [-2,0,-20], [10,-2,4]], dtype=np.float32)
display(A)

b = np.array([1,1,1], dtype=np.float32)
display(b)

gaussEliminPP(A,b)

array([[  0.,  -2.,   2.],
       [ -2.,   0., -20.],
       [ 10.,  -2.,   4.]], dtype=float32)

array([1., 1., 1.], dtype=float32)

array([ 0.01020409, -0.5510204 , -0.05102041], dtype=float32)